In [1]:
import os

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from lxml import etree # type: ignore <- pylance milně hlásí chybu
from pathlib import Path
import time
import sys

from utils import *

os.chdir(r'E:\CVUT_BAP')

In [48]:
namespaces = {
    'm': 'istp:opendata:schemas:MereniSeznam:v1', 
    'd': 'istp:opendata:schemas:DatovaSada:v1'      
}

xml_file = Path(r'kod\data\priklady\data_z_mericich_pristoroju\Data z měřících přístrojů 28-08-2021.xml')
tree = etree.parse(xml_file)
datovy_obsah = tree.find('d:DatovyObsah', namespaces)
mereni_seznam_element = datovy_obsah[0]

In [49]:
len(mereni_seznam_element)

447

In [144]:
def safe_get(element, xpath, namespaces):
    if element is None:
        return None
    node = element.find(xpath, namespaces)
    return node.text if node is not None else None


def safe_find(element, xpath, namespaces):
    if element is None:
        return None
    return element.find(xpath, namespaces)


def safe_findall(element, xpath, namespaces):
    if element is None:
        return None
    return element.findall(xpath, namespaces)


def safe_get_attribute(element, attribute_name):
    if element is None:
        return None
    return element.get(attribute_name)


def get_poznamky(element, namespaces):
    poznamky = safe_findall(element, 'm:poznamka', namespaces)
    if poznamky is None:
        return None
    return '\n'.join(p.text for p in poznamky)


def get_result_attributes(element, prefix):
    return {
        f'{prefix}_Hodnota': safe_get_attribute(element, 'hodnota'),
        f'{prefix}_Vysledek': safe_get_attribute(element, 'vysledek')
    }


def get_boundary_attributes(element, prefix):
    return {
        f'{prefix}_Hodnota': safe_get_attribute(element, 'hodnota'),
        f'{prefix}_RucniZadani': safe_get_attribute(element, 'rucniZadani')
    }


def get_measured(element, prefix, namespaces):
    min = safe_find(element, 'm:min', namespaces)
    max = safe_find(element, 'm:max', namespaces)

    n_attributes = get_result_attributes(element, prefix)
    min_attributes = get_boundary_attributes(min, f'{prefix}_Min')
    max_attributes = get_boundary_attributes(max, f'{prefix}_Max')
    
    return n_attributes | min_attributes | max_attributes
    


def get_monitor_attributes(element, prefix):
    return {
        f'{prefix}_Podporovano': safe_get_attribute(element, 'podporovano'),
        f'{prefix}_Otestovano': safe_get_attribute(element, 'otestovano')
    }


def get_otacky_benzin(element, prefix, namespaces):
    measured_list = ['CO', 'CO2', 'COCOOR', 'HC', 'LAMBDA', 'N', 'NOX', 'O2', 'TPS']
    return {key: value for measured in measured_list for key, value in get_measured(safe_find(element, f'm:{measured}', namespaces), f'{prefix}_{measured}', namespaces).items()}





mereni1 = mereni_seznam_element[0]

In [ ]:
emise_record = {}
emise_record['CisloProtokolu'] = safe_get(mereni1, 'm:CisloProtokolu', namespaces)
emise_record['DatumProhlidky'] = safe_get(mereni1, 'm:DatumProhlidky', namespaces)
emise_record['StaniceCislo'] = safe_get(safe_find(mereni1, 'm:Stanice', namespaces), 'm:Cislo', namespaces)
emise_record['Zahajeni'] = safe_get(safe_find(mereni1, 'm:CasoveUdaje', namespaces), 'm:Zahajeni', namespaces)
emise_record['Ukonceni'] = safe_get(safe_find(mereni1, 'm:CasoveUdaje', namespaces), 'm:Ukonceni', namespaces)
emise_record['OdpovednaOsoba'] = safe_get(mereni1, 'm:OdpovednaOsoba', namespaces)

pristroj_data_element = safe_find(mereni1, 'm:PristrojData', namespaces)
prohlidka_element = safe_find(pristroj_data_element, 'm:prohlidka', namespaces)
emise_record['Prohlidka_CisloProtokolu'] = safe_get_attribute(prohlidka_element, 'cisloProtokolu')
emise_record['Prohlidka_DatumProhlidky'] = safe_get_attribute(prohlidka_element, 'datumProhlidky')

merici_pristroj_element = safe_find(prohlidka_element, 'm:mericiPristroj', namespaces)
emise_record['MericiPristroj_Vyrobce'] = safe_get_attribute(merici_pristroj_element, 'vyrobce')
emise_record['MericiPristroj_Typ'] = safe_get_attribute(merici_pristroj_element, 'typ')
emise_record['MericiPristroj_Verze'] = safe_get_attribute(merici_pristroj_element, 'verze')
emise_record['MericiPristroj_OBD'] = safe_get_attribute(merici_pristroj_element, 'OBD')
emise_record['MericiPristroj_VerzeSoftware'] = safe_get_attribute(merici_pristroj_element, 'verzeSoftware')

emise_record['Poznamky'] = get_poznamky(prohlidka_element, namespaces)

vozidlo_element = safe_find(pristroj_data_element, 'm:vozidlo', namespaces)
emise_record['Vozidlo_Vin'] = safe_get(vozidlo_element, 'm:VIN', namespaces)
emise_record['Vozidlo_Znacka'] = safe_get(vozidlo_element, 'm:tovarniZnacka', namespaces)
emise_record['Vozidlo_ObchodniOznaceni'] = safe_get(vozidlo_element, 'm:typVozidla', namespaces)
emise_record['Vozidlo_TypMotoru'] = safe_get(vozidlo_element, 'm:typMotoru', namespaces)
emise_record['Vozidlo_CisloMotoru'] = safe_get(vozidlo_element, 'm:cisloMotoru', namespaces)
emise_record['Vozidlo_Odometer'] = safe_get(vozidlo_element, 'm:stavTachometru', namespaces)
emise_record['Vozidlo_RokVyroby'] = safe_get(vozidlo_element, 'm:rokVyroby', namespaces)
emise_record['Vozidlo_DatumPrvniRegistrace'] = safe_get(vozidlo_element, 'm:datumPrvniRegistrace', namespaces)
emise_record['Vozidlo_Palivo'] = safe_get(vozidlo_element, 'm:palivo', namespaces)

vysledek_mereni_element = safe_find(pristroj_data_element, 'm:vysledekMereni', namespaces)
emise_record['Vysledek_VisualniKontrola'] = safe_get_attribute(vysledek_mereni_element, 'vysledekVisualniKontroly')
emise_record['Vysledek_Readiness'] = safe_get_attribute(vysledek_mereni_element, 'vysledekReadiness')
emise_record['Vysledek_RidiciJednotka'] = safe_get_attribute(vysledek_mereni_element, 'vysledekRidiciJednotka')
emise_record['Vysledek_RidiciJednotkaStav'] = safe_get_attribute(vysledek_mereni_element, 'vysledekRidiciJednotkaStav')
emise_record['Vysledek_Mil'] = safe_get_attribute(vysledek_mereni_element, 'vysledekMIL')

vyhovuje_element = safe_find(vysledek_mereni_element, 'm:vyhovuje', namespaces)
emise_record['Vysledek_Vyhovuje'] = 'true' if vyhovuje_element is not None else None
emise_record['PristiProhlidka'] = safe_get_attribute(vyhovuje_element, 'pristiProhlidka')

emisni_system_element = safe_find(pristroj_data_element, 'm:emisniSystem', namespaces)
rizeny_obd_element = safe_find(emisni_system_element, 'm:rizenyOBD', namespaces)
rizeny_element = safe_find(emisni_system_element, 'm:rizeny', namespaces)
nerizeny_element = safe_find(emisni_system_element, 'm:nerizeny', namespaces)
emise_record['EmisniSystem'] = "Rizeny_Obd" if rizeny_obd_element is not None else "Rizeny" if rizeny_element is not None else "Nerizeny" if nerizeny_element is not None else None
emise_record['Obd_KomunikacniProtokol'] = safe_get(rizeny_obd_element, 'm:komunikacniProtokol', namespaces)
emise_record['Obd_Vin'] = safe_get(rizeny_obd_element, 'm:VIN', namespaces)
emise_record['Obd_PocetDtc'] = safe_get(rizeny_obd_element, 'm:pocetDTC', namespaces) if not None else safe_get(rizeny_element, 'm:pocetDTC', namespaces)
emise_record['Obd_VzdalenostDtc'] = safe_get(rizeny_obd_element, 'm:vzdalenostDTC', namespaces)
emise_record['Obd_CasDtc'] = safe_get(rizeny_obd_element, 'm:casDTC', namespaces)
emise_record['Obd_KontrolaMil'] = safe_get(rizeny_obd_element, 'm:kontrolaMIL', namespaces)

readiness_element = safe_find(rizeny_obd_element, 'm:readiness', namespaces)
emise_record['Obd_Readiness_Vysledek'] = safe_get_attribute(readiness_element, 'vysledek')

all_monitors = [
    ('Zazeh', 'OBDzazeh', ['AC', 'CAT-FUNC', 'COMP', 'EGR-VVT', 'EVAP', 'FUEL', 'HCAT', 'MISF', 'O2S-FUNC', 'O2S-HEAT', 'SAS']),
    ('Vznet', 'OBDvznet', ['AC', 'BOOST', 'COMP', 'DPF', 'EGR-VVT', 'EGS', 'FUEL', 'MISF', 'NMHC', 'NOX', 'RESERVE']),
    ('J1939', 'J1939', ['AC', 'BOOST', 'CAT-FUNC', 'COLD', 'COMP', 'DPF', 'EGR-VVT', 'EGS-FUNC', 'EGS-HEAT', 'EVAP', 'FUEL', 'HCAT', 'MISF', 'NM-HC', 'NOX', 'SAS'])
]
for col_name, element_name, monitor_list in all_monitors:
    parent = safe_find(readiness_element, f'm:{element_name}', namespaces)
    prefix = f'Obd_Readiness_{col_name}'
    emise_record.update({key: value for monitor in monitor_list for key, value in get_monitor_attributes(safe_find(parent, f'm:{monitor}', namespaces), f'{prefix}_{monitor}').items()})

detal_benzin_element = safe_find(pristroj_data_element, 'm:detailBenzin', namespaces)
emise_record['Benzin_Palivo'] = safe_get_attribute(detal_benzin_element, 'palivo')

benzin_vyusteni_element_list = safe_findall(detal_benzin_element, 'm:vyusteni', namespaces)
if benzin_vyusteni_element_list is not None:
    for i in range(4):
        if i < len(benzin_vyusteni_element_list):
            benzin_vyusteni_element = benzin_vyusteni_element_list[i]
        else:
            benzin_vyusteni_element = None
        emise_record.update(get_otacky_benzin(safe_find(benzin_vyusteni_element, 'm:otackyVolnobezne', namespaces), f'Benzin_Vyusteni{i}_OtackyVolnobezne', namespaces))
        emise_record.update(get_otacky_benzin(safe_find(benzin_vyusteni_element, 'm:otackyZvysene', namespaces), f'Benzin_Vyusteni{i}_OtackyZvysene', namespaces))







display(emise_record)

{'CisloProtokolu': 'CZ-003237-21-08-1140',
 'DatumProhlidky': '2021-08-28',
 'StaniceCislo': '3237',
 'Zahajeni': '2021-08-28T12:07:59.3730000+02:00',
 'Ukonceni': '2021-08-28T12:17:24.1300000+02:00',
 'OdpovednaOsoba': '49493',
 'Prohlidka_CisloProtokolu': 'CZ-003237-21-08-1140',
 'Prohlidka_DatumProhlidky': '2021-08-28T12:16:00',
 'MericiPristroj_Vyrobce': 'ActiaCZ',
 'MericiPristroj_Typ': 'AT505',
 'MericiPristroj_Verze': '-',
 'MericiPristroj_OBD': 'R+OBD',
 'MericiPristroj_VerzeSoftware': '2.08.3',
 'Poznamky': 'Detailní výpis výsledků kontroly emisí je uveden v příloze tohoto protokolu\nTyp motoru instalovaného ve vozidle souhlasí s typem motoru uvedeným v dokladech',
 'Vozidlo_Vin': 'VF3GJNFUC95147274',
 'Vozidlo_Znacka': None,
 'Vozidlo_ObchodniOznaceni': 'PARTNER',
 'Vozidlo_TypMotoru': 'NFU',
 'Vozidlo_CisloMotoru': '-',
 'Vozidlo_Odometer': '217811',
 'Vozidlo_RokVyroby': '2005',
 'Vozidlo_DatumPrvniRegistrace': '2005-04-22',
 'Vozidlo_Palivo': 'BA',
 'Vysledek_VisualniKontr

In [132]:
benzin_vyusteni_element